In [1]:
import nltk
import itertools
from nltk.tokenize import TweetTokenizer
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import enchant
from collections import Counter
import seaborn as sns
from nltk.corpus import stopwords
%matplotlib inline

- arrangeTweet Takes a tweet as Input and normalize each word
- for example : 'llooooooovvvee' becomes 'love'

In [2]:
d = enchant.Dict("en_US")
def arrangeTweet(tweet):
    tweet=tweet.lower()
    tweet=tweet.split()
    for i in range(len(tweet)):
        if d.check(''.join(''.join(s)[:2] for _, s in itertools.groupby(tweet[i]))):
            tweet[i]=''.join(''.join(s)[:2] for _, s in itertools.groupby(tweet[i]))
        else:
            tweet[i]=''.join(''.join(s)[:1] for _, s in itertools.groupby(tweet[i]))
    tweet=' '.join(tweet)
    return tweet

- Cleaning of the tweets using regular expressions

In [3]:
stopwordlist = set(stopwords.words("english"))
def clean(tweet):
    tweet = re.sub(r"\'s", " \'s", tweet)
    tweet = re.sub(r"\'ve", " \'ve", tweet)
    tweet = re.sub(r"n\'t", " n\'t", tweet)
    tweet = re.sub(r"\'re", " \'re", tweet)
    tweet = re.sub(r"\'d", " \'d", tweet)
    tweet = re.sub(r"\'ll", " \'ll", tweet)
    tweet = re.sub(r",", " , ", tweet)
    tweet = re.sub(r"!", " ! ", tweet)
    tweet = re.sub(r"\(", " \( ", tweet)
    tweet = re.sub(r"\)", " \) ", tweet)
    tweet = re.sub(r"\?", " \? ", tweet)
    tweet = re.sub(r"\s{2,}", " ", tweet)
    tweet = arrangeTweet(tweet) 
    liste=[word for word in tweet.split() if word not in stopwordlist]
    # Taking the tweets that have length larger than 1
    liste=[word for word in liste if len(word)>1]
    tweet=' '.join(liste)
    return tweet.strip().lower()

- Below, we clean the files for the small dataset since we don't have enough memory to build TF-IDF for the full dataset

In [5]:
oldfile=open("train_pos.txt",'rb')
sentences=[]
i=0
for line in oldfile:
    if i%30000==0:
        print(i)
    i=i+1
    
    line = line.decode('utf8')
    line=clean(line)
    #if not (line in sentences):
    sentences.append(line)

0
30000
60000
90000


In [ ]:
Counter(x)['love']

In [7]:
oldfile=open("train_neg.txt",'rb')
i=0
for line in oldfile:
    if i%30000==0:
        print(i)
    i+=1
    line = line.decode('utf8')
    line=clean(line)
    #if not (line in sentences):
    sentences.append(line)

0
30000
60000
90000


- Shuffling of the indices

In [7]:
indexes=np.arange(0,200000)
np.random.shuffle(indexes)

In [8]:
y=[0]*100000
y2=[1]*100000
y=y+y2
y=np.array(y)
y=y[indexes]

- Using Scikit-learn we built the TF-IDF matrix

In [9]:
def scikit_TFIDF(m,n,Total_clean_train,Total_clean_test):
    vectorizer = CountVectorizer(min_df=1,max_features=m,ngram_range=(1,n))
    analyze = vectorizer.build_analyzer
    X_train = vectorizer.fit_transform(Total_clean_train).toarray()
    X_test = vectorizer.transform(Total_clean_test).toarray()
    transformer = TfidfTransformer()
    tfidf_train=transformer.fit_transform(X_train).toarray()
    tfidf_test=transformer.transform(X_test).toarray()
    return tfidf_train,tfidf_test

In [10]:
testfile=open("test_data.txt",'rb')
i=0
sentences_test=[]
for line in testfile:
    if i%5000==0:
        print(i)
    i=i+1
    line=line.decode('utf8')
    line=clean(line)
    sentences_test.append((line[line.find(',')+1:]))

0
5000


In [11]:
X_train,X_test=scikit_TFIDF(12000 ,2,sentences,sentences_test)

In [12]:
len(sentences)

200000

In [13]:
X_train=X_train[indexes]

In [14]:
test_x=X_train[170000:]
test_y=y[170000:]

## LOGISTIC REGRESSION

In [15]:
from sklearn import linear_model
logisticReg = linear_model.LogisticRegression(multi_class='ovr')
logisticReg.fit(X_train[0:169999], y[0:169999])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [16]:
Y_predict=logisticReg.predict(test_x)

In [17]:
print(classification_report(Y_predict,test_y))

             precision    recall  f1-score   support

          0       0.82      0.79      0.81     15668
          1       0.78      0.81      0.80     14332

avg / total       0.80      0.80      0.80     30000



# SVM

In [ ]:
from sklearn import svm

In [ ]:
lin_clf = svm.LinearSVC()

In [ ]:
lin_clf.fit(X_train[0:169999], y[0:169999])

In [ ]:
Y_SVC_linear_predic=lin_clf.predict(test_x)

In [ ]:
print(classification_report(Y_SVC_linear_predic,test_y))

# BAYES

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
gnb = GaussianNB()

In [ ]:
Y_pred_Bayes = gnb.fit(X_train[0:169999], y[0:169999]).predict(test_x)

In [ ]:
print(classification_report(Y_pred_Bayes,test_y))

#   BUILDING OF LIGHTGBM                      

- Building the files in the format of LightGBM

In [ ]:
file=open("binary.train",'w')
i=0
for i in range(len(y[0:169999])):
    if i%10000==0:
        print(i)
    i=i+1
    line=str(y[i])+"\t"+"\t".join([str(x[0]) for x in np.split(X_train[i,:],len(X_train[0,:]))])
    #print(line)
    if i==169999:
        #To prevent the "\n" for the last line
        file.write(line)
    else:
        file.write(line+"\n")

In [ ]:
file=open("binary.test",'w')
i=0
for i in range(len(y[170000:])):
    if i%500==0:
        print(i)
    i=i+1
    line=str(y[i])+"\t"+"\t".join([str(x[0]) for x in np.split(X_train[i,:],len(X_train[0,:]))])
    #print(line)
    if i==30000:
        print("hha")
        file.write(line)
    else:
        print("test")
        file.write(line+"\n")
    

In [ ]:
file=open("pred.test",'w')
i=0
for i in range(len(X_test[:,0])):
    if i%500==0:
        print(i)
    line="\t".join([str(x[0]) for x in np.split(X_test[i,:],len(X_test[0,:]))])
    i=i+1
    #print(line)
    if i==len(X_test[:,0]):
        print("hha")
        file.write(line)
    else:
        file.write(line+"\n")

In [ ]:
filen=open("pred.txt","rb")

## Creating the CSV file for Kaggle

In [ ]:
y_pred=[]
for line in filen:
    y_pred.append(int(float(line)>0.5)) 

In [ ]:
y_pred=np.array(y_pred)

In [ ]:
def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in csv format for submission to kaggle
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, 'w') as csvfile:
        fieldnames = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({'Id':int(r1),'Prediction':int(r2)})

In [ ]:
y_pred=1-2*y_pred

In [ ]:
import csv

In [ ]:
create_csv_submission(np.arange(1,10001),y_pred,'predic.csv')